# Phase 0.3 — Bot-IoT Dataset: Deep Data Understanding (Read-Only EDA)

> **PHASE 0 CONTRACT — READ-ONLY**
> This notebook performs **zero mutations** on the source data.
> No rows are dropped, no columns are altered, no imputation is performed.
> All operations are purely observational. The kernel can be re-run from scratch at any time and will produce identical output.

---

## Dataset Overview

| Property | Value |
|---|---|
| **Dataset** | UNSW Bot-IoT (2018) |
| **Source** | University of New South Wales Canberra Cyber Range |
| **Format** | Standard CSV (no metadata headers) |
| **Files** | 4 × `UNSW_2018_IoT_Botnet_Full5pc_*.csv` |
| **Approx. rows** | ~3.5M per file (~14M total) |
| **Features** | 46 columns (flow stats + UNSW-engineered behavioral features) |
| **Labels** | Tri-label: `attack` (binary) + `category` + `subcategory` |
| **Attack types** | DoS, DDoS, OS Scan, Keylogging, Data Exfiltration |

---

## Cross-Dataset Comparison: TON-IoT vs IoT-23 vs Bot-IoT

| Dimension | TON-IoT | IoT-23 (CTU) | Bot-IoT (UNSW) |
|---|---|---|---|
| **Format** | CSV (engineered) | Zeek TSV (raw) | CSV (engineered) |
| **Features** | 46 cols | 22 Zeek cols | 46 cols |
| **Label type** | Binary + multi-class type | Binary + fine-grained | Binary + category + subcategory |
| **Sentinels** | Numeric zeros as implicit | `-` and `?` (explicit Zeek) | `-1` port (ARP), numeric |
| **Pre-engineering** | Yes (UNSW-style stats) | No (raw conn telemetry) | Yes (UNSW-style stats) |
| **Time column** | `ts` (Unix float) | `ts` (Unix float) | `stime` / `ltime` (Unix float) |
| **Protocol col** | `proto` (string) | `proto` (string) | `proto` + `proto_number` |
| **IP columns** | `src_ip`, `dst_ip` | `id.orig_h`, `id.resp_h` | `saddr`, `daddr` |
| **Port columns** | `src_port`, `dst_port` | `id.orig_p`, `id.resp_p` | `sport`, `dport` |

---

## Sections in This Notebook

| Section | Title |
|---|---|
| **0.1** | File Inventory + Sampling Strategy + Data Loading |
| **0.2** | Column Inventory + Initial Feature Semantics |
| **0.3** | Categorical Column Value Analysis + Sentinel Detection |
| **0.4** | Numerical Column Statistics + Semantic Annotations |
| **0.5** | Full Bot-IoT Feature Meaning Dictionary (Cross-Dataset Bridge) |
| **0.6** | Feature Role Classification (IDS Perspective) |
| **0.7** | Engineering Challenges + Artifact Export + Markdown Report |

---
## Section 0.1 — File Inventory, Loader, and Sampling Strategy

In [1]:
# ============================================================
# SECTION 0.1: Imports, Configuration, File Inventory
# ============================================================
import warnings
warnings.filterwarnings("ignore")

import os
import pathlib
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 60)
pd.set_option("display.max_colwidth", 80)
pd.set_option("display.float_format", "{:.4f}".format)

# ── Notebook directory ────────────────────────────────────────
NOTEBOOK_DIR = pathlib.Path(__file__).parent if "__file__" in dir() else pathlib.Path().resolve()
if "Phase_0" not in str(NOTEBOOK_DIR):
    NOTEBOOK_DIR = pathlib.Path(r"C:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\Phase_0")

# ── Base data path ────────────────────────────────────────────
BASE_DATA_PATH = NOTEBOOK_DIR.parent / "data" / "bot_iot"
if not BASE_DATA_PATH.exists():
    # secondary fallback
    BASE_DATA_PATH = pathlib.Path(r"C:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\bot_iot")

print(f"📁 NOTEBOOK_DIR  : {NOTEBOOK_DIR}")
print(f"📁 BASE_DATA_PATH: {BASE_DATA_PATH}")
print(f"   exists → {BASE_DATA_PATH.exists()}")

# ── Constants ─────────────────────────────────────────────────
MAX_ROWS_PER_FILE = 100_000   # Phase 0 safety cap — prevents OOM
READ_ONLY         = True       # guard constant — never mutate source data
LABEL_COL         = "attack"
CAT_COL           = "category"
SUBCAT_COL        = "subcategory"

BOT_IOT_FILENAMES = [
    "UNSW_2018_IoT_Botnet_Full5pc_1.csv",
    "UNSW_2018_IoT_Botnet_Full5pc_2.csv",
    "UNSW_2018_IoT_Botnet_Full5pc_3.csv",
    "UNSW_2018_IoT_Botnet_Full5pc_4.csv",
]

# ── File inventory ────────────────────────────────────────────
print("\n" + "=" * 65)
print("📋 BOT-IOT FILE INVENTORY")
print("=" * 65)

inv_rows = []
for fname in BOT_IOT_FILENAMES:
    fpath = BASE_DATA_PATH / fname
    size_mb = fpath.stat().st_size / (1024 ** 2) if fpath.exists() else 0
    # Quick row count estimate from file size (first line = header)
    inv_rows.append({
        "file":    fname,
        "exists":  fpath.exists(),
        "size_mb": round(size_mb, 1),
        "path":    str(fpath),
    })

inventory_df = pd.DataFrame(inv_rows)
display(inventory_df[["file", "exists", "size_mb"]])
print(f"\n  Total raw size: {inventory_df['size_mb'].sum():.1f} MB across {len(inventory_df)} files")

📁 NOTEBOOK_DIR  : C:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\Phase_0
📁 BASE_DATA_PATH: C:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\bot_iot
   exists → True

📋 BOT-IOT FILE INVENTORY


,file,exists,size_mb
0,UNSW_2018_IoT_Botnet_Full5pc_1.csv,True,259.7000
1,UNSW_2018_IoT_Botnet_Full5pc_2.csv,True,260.4000
2,UNSW_2018_IoT_Botnet_Full5pc_3.csv,True,269.1000
3,UNSW_2018_IoT_Botnet_Full5pc_4.csv,True,180.8000



  Total raw size: 970.0 MB across 4 files


### Why `MAX_ROWS_PER_FILE = 100,000`?

| Concern | Explanation |
|---|---|
| **OOM safety** | Each file is ~260–280 MB on disk. Full load of 4 files (~14M rows × 46 cols) can exceed 8–16 GB RAM and crash the kernel. |
| **EDA validity** | Schema, dtypes, sentinel patterns, and value distributions are stable across 100k rows — we don't need all rows to understand the data. |
| **Speed** | 100k rows load in <1s, enabling fast iterative exploration in Phase 0. |
| **Phase 1 distinction** | Phase 1 (preprocessing) will load the **full dataset** using chunked I/O — no cap. |

> **Bot-IoT vs IoT-23:** Bot-IoT uses a standard CSV format with a single header row.  
> No Zeek-style metadata headers, no `#fields` line, no `#close` footer. Pure tabular data.

In [2]:
# ── CSV loader ────────────────────────────────────────────────
def load_botiot_csv(filepath: pathlib.Path, max_rows: int = MAX_ROWS_PER_FILE) -> pd.DataFrame:
    """
    Load up to `max_rows` rows from a Bot-IoT CSV file.
    
    Bot-IoT format:
      - Standard CSV, comma-separated
      - Row 0 = header (column names)
      - No metadata lines, no footer sentinels
      - Low-memory=False for consistent dtype inference
    """
    df = pd.read_csv(
        filepath,
        nrows=max_rows,
        low_memory=False,
    )
    df["source_file"] = filepath.name   # track provenance
    return df

# ── Load all files ────────────────────────────────────────────
print("=" * 65)
print("📥 LOADING BOT-IOT FILES (Phase 0 sample: 100k rows/file)")
print("=" * 65)

frames = []
for fname in BOT_IOT_FILENAMES:
    fpath = BASE_DATA_PATH / fname
    if not fpath.exists():
        print(f"  ⚠️  MISSING: {fname} — skipping")
        continue
    chunk = load_botiot_csv(fpath)
    frames.append(chunk)
    print(f"  ✅ {fname:50s}  →  {len(chunk):>7,} rows  ×  {chunk.shape[1]} cols")

assert frames, "No files loaded — check BASE_DATA_PATH!"

df = pd.concat(frames, ignore_index=True)
total_rows = len(df)
total_cols = len(df.columns) - 1   # exclude source_file
mem_mb     = df.memory_usage(deep=True).sum() / (1024 ** 2)

print(f"\n{'─'*65}")
print(f"  Combined DataFrame : {total_rows:,} rows  ×  {df.shape[1]} columns")
print(f"  Memory usage       : {mem_mb:.1f} MB")
print(f"  Files sampled      : {len(frames)} / {len(BOT_IOT_FILENAMES)}")
print(f"  MAX_ROWS_PER_FILE  : {MAX_ROWS_PER_FILE:,}  (Phase 0 cap)")
print(f"{'─'*65}")

📥 LOADING BOT-IOT FILES (Phase 0 sample: 100k rows/file)
  ✅ UNSW_2018_IoT_Botnet_Full5pc_1.csv                  →  100,000 rows  ×  47 cols
  ✅ UNSW_2018_IoT_Botnet_Full5pc_2.csv                  →  100,000 rows  ×  47 cols
  ✅ UNSW_2018_IoT_Botnet_Full5pc_3.csv                  →  100,000 rows  ×  47 cols
  ✅ UNSW_2018_IoT_Botnet_Full5pc_4.csv                  →  100,000 rows  ×  47 cols

─────────────────────────────────────────────────────────────────
  Combined DataFrame : 400,000 rows  ×  47 columns
  Memory usage       : 297.7 MB
  Files sampled      : 4 / 4
  MAX_ROWS_PER_FILE  : 100,000  (Phase 0 cap)
─────────────────────────────────────────────────────────────────


---
## Section 0.2 — Column Inventory & Initial Feature Semantics

The Bot-IoT dataset contains **46 feature columns** (plus `source_file` added by our loader).
Features span three conceptual layers:

| Layer | Columns | Source |
|---|---|---|
| **Flow identifiers** | `pkSeqID`, `stime`, `ltime`, `saddr`, `sport`, `daddr`, `dport` | Raw network capture |
| **Protocol + State** | `proto`, `proto_number`, `flgs`, `flgs_number`, `state`, `state_number` | Argus/NetFlow parsing |
| **Volume stats** | `pkts`, `bytes`, `spkts`, `dpkts`, `sbytes`, `dbytes` | Per-flow counts |
| **Time stats** | `dur`, `seq`, `mean`, `stddev`, `sum`, `min`, `max` | Per-flow statistics |
| **Rate features** | `rate`, `srate`, `drate` | Computed Bps |
| **UNSW behavioral features** | `TnBP*`, `TnP_*`, `AR_P_*`, `N_IN_*`, `Pkts_P_*` (14 cols) | UNSW-engineered window stats |
| **Labels** | `attack`, `category`, `subcategory` | Expert annotation |

In [3]:
# ============================================================
# SECTION 0.2: Column Inventory + Initial Feature Semantics
# ============================================================

# ── Initial semantics mapping (brief; full dict in Section 0.5) ──
BOTIOT_FEATURE_SEMANTICS = {
    "pkSeqID":                        "Packet sequence ID — row identifier",
    "stime":                          "Flow start time (Unix epoch float)",
    "flgs":                           "Argus flow flags (string, e.g. 'e')",
    "flgs_number":                    "Numeric encoding of flgs",
    "proto":                          "Layer-4 protocol name (tcp/udp/arp/icmp…)",
    "proto_number":                   "Numeric encoding of proto",
    "saddr":                          "Source IP address",
    "sport":                          "Source port (-1 for ARP/non-port protocols)",
    "daddr":                          "Destination IP address",
    "dport":                          "Destination port (-1 for ARP/non-port protocols)",
    "pkts":                           "Total packets (src + dst)",
    "bytes":                          "Total bytes (src + dst)",
    "state":                          "Connection state label (e.g. RST, CON, FIN)",
    "state_number":                   "Numeric encoding of state",
    "ltime":                          "Flow last-packet time (Unix epoch float)",
    "seq":                            "Argus transaction sequence number",
    "dur":                            "Flow duration in seconds",
    "mean":                           "Mean packet IAT (inter-arrival time) or size",
    "stddev":                         "Standard deviation of packet size/IAT",
    "sum":                            "Sum of packet sizes",
    "min":                            "Minimum packet size in flow",
    "max":                            "Maximum packet size in flow",
    "spkts":                          "Source-to-destination packet count",
    "dpkts":                          "Destination-to-source packet count",
    "sbytes":                         "Source-to-destination byte count",
    "dbytes":                         "Destination-to-source byte count",
    "rate":                           "Overall flow rate (bps)",
    "srate":                          "Source flow rate (bps)",
    "drate":                          "Destination flow rate (bps)",
    "TnBPSrcIP":                      "Total bytes from source IP (recent window)",
    "TnBPDstIP":                      "Total bytes to destination IP (recent window)",
    "TnP_PSrcIP":                     "Total packets from source IP (recent window)",
    "TnP_PDstIP":                     "Total packets to destination IP (recent window)",
    "TnP_PerProto":                   "Total packets for this protocol (recent window)",
    "TnP_Per_Dport":                  "Total packets to this destination port (window)",
    "AR_P_Proto_P_SrcIP":             "Avg rate: protocol × source IP",
    "AR_P_Proto_P_DstIP":             "Avg rate: protocol × destination IP",
    "N_IN_Conn_P_DstIP":              "# inbound connections to destination IP (window)",
    "N_IN_Conn_P_SrcIP":              "# inbound connections from source IP (window)",
    "AR_P_Proto_P_Sport":             "Avg rate: protocol × source port",
    "AR_P_Proto_P_Dport":             "Avg rate: protocol × destination port",
    "Pkts_P_State_P_Protocol_P_DestIP": "Packets: state × protocol × destination IP",
    "Pkts_P_State_P_Protocol_P_SrcIP":  "Packets: state × protocol × source IP",
    "attack":                         "Binary label: 0=Normal, 1=Attack",
    "category":                       "Attack category (e.g. DoS, DDoS, Normal)",
    "subcategory":                    "Fine-grained attack subtype",
}

# ── Build column inventory ────────────────────────────────────
print("=" * 65)
print("📋 COLUMN INVENTORY")
print("=" * 65)

inv_rows = []
for col in df.columns:
    if col == "source_file":
        continue
    dtype_str   = str(df[col].dtype)
    null_pct    = df[col].isna().mean() * 100
    n_unique    = df[col].nunique()
    semantics   = BOTIOT_FEATURE_SEMANTICS.get(col, "— no mapping —")
    inv_rows.append({
        "column":   col,
        "dtype":    dtype_str,
        "null_%":   round(null_pct, 2),
        "unique":   n_unique,
        "semantics": semantics,
    })

col_inventory = pd.DataFrame(inv_rows)
display(col_inventory)

# ── Null / zero-null summary ──────────────────────────────────
high_null_cols  = col_inventory[col_inventory["null_%"] > 5]["column"].tolist()
zero_null_cols  = col_inventory[col_inventory["null_%"] == 0]["column"].tolist()
all_null_cols   = col_inventory[col_inventory["null_%"] == 100]["column"].tolist()

print(f"\n  Total columns          : {len(col_inventory)}")
print(f"  Zero-null columns      : {len(zero_null_cols)}")
print(f"  High-null (>5%) cols   : {len(high_null_cols)}  → {high_null_cols}")
print(f"  All-null (100%) cols   : {len(all_null_cols)}  → {all_null_cols}")

📋 COLUMN INVENTORY


,column,dtype,null_%,unique,semantics
0,pkSeqID,int64,0.0000,400000,Packet sequence ID — row identifier
1,stime,float64,0.0000,38946,Flow start time (Unix epoch float)
2,flgs,object,0.0000,6,"Argus flow flags (string, e.g. 'e')"
3,flgs_number,int64,0.0000,6,Numeric encoding of flgs
4,proto,object,0.0000,3,Layer-4 protocol name (tcp/udp/arp/icmp…)
5,proto_number,int64,0.0000,3,Numeric encoding of proto
6,saddr,object,0.0000,8,Source IP address
7,sport,int64,0.0000,65537,Source port (-1 for ARP/non-port protocols)
8,daddr,object,0.0000,8,Destination IP address
9,dport,int64,0.0000,2,Destination port (-1 for ARP/non-port protocols)



  Total columns          : 46
  Zero-null columns      : 46
  High-null (>5%) cols   : 0  → []
  All-null (100%) cols   : 0  → []


---
## Section 0.3 — Categorical Column Value Analysis & Sentinel Detection

Bot-IoT uses standard CSV — no Zeek `-` or `?` sentinels. However, it has its own quirks:

| Sentinel / Pattern | Meaning | Columns Affected |
|---|---|---|
| `-1` in numeric | ARP / non-port protocol (Argus convention) | `sport`, `dport` |
| `0` bytes | One-way flow or no response | `dbytes`, `dpkts`, `sbytes` |
| `0.0` duration | Sub-millisecond flow or ICMP probe | `dur` |
| Empty string `""` | Rarer; may appear in `state` after join | `state` |
| `NaN` | Genuine missing (parser artifact) | Various |

This section tracks: empty string, `NaN`, and special string values per column.

In [4]:
# ============================================================
# SECTION 0.3: Categorical Column Value Analysis
# ============================================================

categorical_columns = [c for c in df.columns
                        if df[c].dtype == object and c != "source_file"]
numerical_columns   = [c for c in df.columns
                        if df[c].dtype in ["int64","float64","int32","float32"]
                        and c != "source_file"]

print("=" * 65)
print("📋 COLUMN TYPE BREAKDOWN")
print("=" * 65)
print(f"  Categorical (object)  : {len(categorical_columns)}  → {categorical_columns}")
print(f"  Numerical (int/float) : {len(numerical_columns)}  → {numerical_columns}")

# ── Per-column value analysis ─────────────────────────────────
def analyze_categorical_column(df_in, col, max_display=30):
    """Return value counts with semantic interpretation for Bot-IoT."""
    col_data = df_in[col]
    total    = len(col_data)
    vc       = col_data.value_counts(dropna=False)
    rows     = []
    for val, cnt in vc.items():
        pct   = cnt / total * 100
        v_str = str(val) if not pd.isna(val) else "<NaN>"
        if v_str == "<NaN>":
            interpretation = "NULL: pandas NaN (true missing)"
        elif v_str in ("", "None", "null"):
            interpretation = "PLACEHOLDER: Empty / null string"
        elif v_str == "(empty)":
            interpretation = "PLACEHOLDER: Empty field sentinel"
        elif v_str == "0":
            interpretation = "DATA: Zero (may indicate no traffic / one-way flow)"
        elif v_str == "-1":
            interpretation = "PLACEHOLDER: No port (ARP / non-port protocol)"
        else:
            interpretation = "DATA: Actual value"
        rows.append({"Value": v_str, "Count": int(cnt), "Pct": round(pct, 2),
                     "Interpretation": interpretation})
    result = pd.DataFrame(rows)
    return (result if len(result) <= max_display else result.head(max_display)), len(vc)

print("\n" + "=" * 65)
print("🔤 CATEGORICAL COLUMN VALUE ANALYSIS")
print("=" * 65)

cat_analyses = {}
for col in categorical_columns:
    print(f"\n{'─'*65}")
    print(f"📊  COLUMN: {col}")
    print(f"{'─'*65}")
    analysis_df, n_unique = analyze_categorical_column(df, col)
    cat_analyses[col] = {"df": analysis_df, "n_unique": n_unique}
    if n_unique > 30:
        print(f"   ⚠️ High cardinality: {n_unique} unique values (showing top 30)")
    display(analysis_df)

# ── Comprehensive sentinel summary ───────────────────────────
print("\n" + "=" * 65)
print("📊 COMPREHENSIVE SENTINEL SUMMARY ACROSS ALL CATEGORICAL COLUMNS")
print("=" * 65)

sent_rows = []
for col in categorical_columns:
    ser  = df[col]
    s    = ser.astype(str)
    row  = {"Column": col, "Total": len(ser)}
    row["empty"]     = int((s == "").sum())
    row["(empty)"]   = int((s == "(empty)").sum())
    row["NaN"]       = int(ser.isna().sum())
    sent_rows.append(row)

sentinel_df = pd.DataFrame(sent_rows)
sentinel_df["any_sentinel"] = sentinel_df[["empty", "(empty)", "NaN"]].sum(axis=1)
sentinel_df = sentinel_df.sort_values("any_sentinel", ascending=False).reset_index(drop=True)
display(sentinel_df)

print("\n📝 KEY INSIGHTS:")
print("   • Bot-IoT uses no Zeek '-' or '?' sentinels — standard CSV format")
print("   • NaN = genuine missing (rare; likely a CSV parse or join artifact)")
print("   • (empty) = explicit empty-field sentinel (rare in this dataset)")
print("   • Numeric -1 in sport/dport = ARP / non-port protocol (check Section 0.4)")

📋 COLUMN TYPE BREAKDOWN
  Categorical (object)  : 7  → ['flgs', 'proto', 'saddr', 'daddr', 'state', 'category', 'subcategory']
  Numerical (int/float) : 39  → ['pkSeqID', 'stime', 'flgs_number', 'proto_number', 'sport', 'dport', 'pkts', 'bytes', 'state_number', 'ltime', 'seq', 'dur', 'mean', 'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP', 'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport', 'Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP', 'attack']

🔤 CATEGORICAL COLUMN VALUE ANALYSIS

─────────────────────────────────────────────────────────────────
📊  COLUMN: flgs
─────────────────────────────────────────────────────────────────


,Value,Count,Pct,Interpretation
0,e,199684,49.9200,DATA: Actual value
1,e s,185187,46.3000,DATA: Actual value
2,e g,12998,3.2500,DATA: Actual value
3,eU,1930,0.4800,DATA: Actual value
4,e *,192,0.0500,DATA: Actual value
5,e d,9,0.0000,DATA: Actual value



─────────────────────────────────────────────────────────────────
📊  COLUMN: proto
─────────────────────────────────────────────────────────────────


,Value,Count,Pct,Interpretation
0,udp,199999,50.0000,DATA: Actual value
1,tcp,199992,50.0000,DATA: Actual value
2,arp,9,0.0000,DATA: Actual value



─────────────────────────────────────────────────────────────────
📊  COLUMN: saddr
─────────────────────────────────────────────────────────────────


...


─────────────────────────────────────────────────────────────────
📊  COLUMN: subcategory
─────────────────────────────────────────────────────────────────


,Value,Count,Pct,Interpretation
0,UDP,200000,50.0000,DATA: Actual value
1,TCP,198515,49.6300,DATA: Actual value
2,HTTP,1485,0.3700,DATA: Actual value



📊 COMPREHENSIVE SENTINEL SUMMARY ACROSS ALL CATEGORICAL COLUMNS


,Column,Total,empty,(empty),NaN,any_sentinel
0,flgs,400000,0,0,0,0
1,proto,400000,0,0,0,0
2,saddr,400000,0,0,0,0
3,daddr,400000,0,0,0,0
4,state,400000,0,0,0,0
5,category,400000,0,0,0,0
6,subcategory,400000,0,0,0,0



📝 KEY INSIGHTS:
   • Bot-IoT uses no Zeek '-' or '?' sentinels — standard CSV format
   • NaN = genuine missing (rare; likely a CSV parse or join artifact)
   • (empty) = explicit empty-field sentinel (rare in this dataset)
   • Numeric -1 in sport/dport = ARP / non-port protocol (check Section 0.4)


---
## Section 0.4 — Numerical Column Statistics & Semantic Annotations

Bot-IoT numerical columns arrive as proper `int64` / `float64` — no string pollution like IoT-23.
Key things to watch:

| Column Group | Expected Min | Expected Max | IDS Relevance |
|---|---|---|---|
| `sport`, `dport` | **-1** (ARP) | 65535 | -1 = ARP — handle separately; cannot scale with ports |
| `dur` | 0.0 | large | 0 = sub-ms flows; extreme = long TCP sessions |
| `sbytes`, `dbytes` | 0 | very large | 0 = one-way / unanswered connection |
| `rate`, `srate`, `drate` | 0 | very large | Extreme values flag DoS/DDoS floods |
| UNSW behavioral features | 0 | large | Window-aggregated; will need normalization in Phase 1 |

In [5]:
# ============================================================
# SECTION 0.4: Numerical Column Statistics + Semantic Annotations
# ============================================================

BOTIOT_NUMERIC_FIELDS = [
    "pkts", "bytes", "spkts", "dpkts", "sbytes", "dbytes",
    "sport", "dport", "dur", "mean", "stddev", "sum", "min", "max",
    "rate", "srate", "drate",
    "TnBPSrcIP", "TnBPDstIP", "TnP_PSrcIP", "TnP_PDstIP",
    "TnP_PerProto", "TnP_Per_Dport",
    "AR_P_Proto_P_SrcIP", "AR_P_Proto_P_DstIP",
    "N_IN_Conn_P_DstIP", "N_IN_Conn_P_SrcIP",
    "AR_P_Proto_P_Sport", "AR_P_Proto_P_Dport",
    "Pkts_P_State_P_Protocol_P_DestIP", "Pkts_P_State_P_Protocol_P_SrcIP",
    "flgs_number", "proto_number", "state_number", "seq", "pkSeqID",
]

# Use a temporary numeric cast dict (READ-ONLY: df is never changed)
num_cast = {}
for fn in BOTIOT_NUMERIC_FIELDS:
    if fn in df.columns:
        num_cast[fn] = pd.to_numeric(df[fn], errors="coerce")

stats_rows = []
for col, series in num_cast.items():
    stats_rows.append({
        "column": col,
        "dtype":  str(df[col].dtype),
        "count":  int(series.count()),
        "mean":   round(float(series.mean()), 4) if series.count() > 0 else None,
        "std":    round(float(series.std()),  4) if series.count() > 0 else None,
        "min":    round(float(series.min()),  4) if series.count() > 0 else None,
        "p25":    round(float(series.quantile(0.25)), 4) if series.count() > 0 else None,
        "median": round(float(series.median()), 4) if series.count() > 0 else None,
        "p75":    round(float(series.quantile(0.75)), 4) if series.count() > 0 else None,
        "max":    round(float(series.max()),  4) if series.count() > 0 else None,
        "null_count": int(series.isna().sum()),
    })

num_stats_df = pd.DataFrame(stats_rows)
print("=" * 65)
print("📊 NUMERICAL COLUMN STATISTICS (temp cast — df unchanged)")
print("=" * 65)
display(num_stats_df)

# ── Semantic annotations ──────────────────────────────────────
BOTIOT_FIELD_SEMANTICS = {
    "pkts":      {"meaning": "Total packets (src+dst)", "zero_means": "No traffic captured", "extreme_means": "High-volume attack", "ids_relevance": "HIGH"},
    "bytes":     {"meaning": "Total bytes (src+dst)",   "zero_means": "No payload",          "extreme_means": "Data exfiltration / flood", "ids_relevance": "HIGH"},
    "spkts":     {"meaning": "Src→dst packet count",    "zero_means": "One-way: dst only",   "extreme_means": "Flood from src", "ids_relevance": "HIGH"},
    "dpkts":     {"meaning": "Dst→src packet count",    "zero_means": "No response",         "extreme_means": "Reverse flood",  "ids_relevance": "HIGH"},
    "sbytes":    {"meaning": "Src→dst byte count",      "zero_means": "No src payload",      "extreme_means": "Large upload",   "ids_relevance": "HIGH"},
    "dbytes":    {"meaning": "Dst→src byte count",      "zero_means": "No response payload", "extreme_means": "Large download", "ids_relevance": "HIGH"},
    "sport":     {"meaning": "Source port (-1=ARP)",    "zero_means": "Port 0 probe",        "extreme_means": "N/A",            "ids_relevance": "MEDIUM — categorical, not ordinal"},
    "dport":     {"meaning": "Dest port (-1=ARP)",      "zero_means": "Port 0 probe",        "extreme_means": "N/A",            "ids_relevance": "MEDIUM — categorical, not ordinal"},
    "dur":       {"meaning": "Flow duration (sec)",     "zero_means": "Sub-millisecond flow","extreme_means": "Long-lived conn", "ids_relevance": "HIGH"},
    "mean":      {"meaning": "Mean packet interval/size","zero_means": "All pkts same size",  "extreme_means": "Variable traffic","ids_relevance": "MEDIUM"},
    "stddev":    {"meaning": "Std dev of packet size",  "zero_means": "Uniform traffic",     "extreme_means": "Bursty / variable","ids_relevance": "MEDIUM"},
    "rate":      {"meaning": "Overall flow rate (bps)", "zero_means": "No traffic",          "extreme_means": "DoS/DDoS flood",  "ids_relevance": "HIGH"},
    "srate":     {"meaning": "Source flow rate (bps)",  "zero_means": "No src traffic",      "extreme_means": "Src-side flood",  "ids_relevance": "HIGH"},
    "drate":     {"meaning": "Dest flow rate (bps)",    "zero_means": "No dst response",     "extreme_means": "Dst-side flood",  "ids_relevance": "HIGH"},
    "TnBPSrcIP": {"meaning": "Total bytes from src IP (window)","zero_means": "New src IP",  "extreme_means": "Src flooding",    "ids_relevance": "HIGH — UNSW behavioral"},
    "TnBPDstIP": {"meaning": "Total bytes to dst IP (window)",  "zero_means": "New dst IP",  "extreme_means": "Dst under attack","ids_relevance": "HIGH — UNSW behavioral"},
    "N_IN_Conn_P_DstIP": {"meaning": "# inbound conns to dst IP","zero_means": "First conn", "extreme_means": "DDoS target",     "ids_relevance": "HIGH"},
    "N_IN_Conn_P_SrcIP": {"meaning": "# inbound conns from src","zero_means": "First conn",  "extreme_means": "Botnet behavior", "ids_relevance": "HIGH"},
}

print("\n" + "=" * 65)
print("📖 SEMANTIC ANNOTATIONS FOR KEY NUMERICAL FIELDS")
print("=" * 65)
for col, sem in BOTIOT_FIELD_SEMANTICS.items():
    if col in num_cast:
        print(f"\n  📌 {col}")
        print(f"     Meaning     : {sem['meaning']}")
        print(f"     Zero means  : {sem['zero_means']}")
        print(f"     Extreme means: {sem['extreme_means']}")
        print(f"     IDS relevance: {sem['ids_relevance']}")

📊 NUMERICAL COLUMN STATISTICS (temp cast — df unchanged)


,column,dtype,count,mean,std,min,p25,median,p75,max,null_count
0,pkts,int64,400000,8.1158,3.3516,2.0000,5.0000,7.0000,11.0000,56.0000,0
1,bytes,int64,400000,702.0735,178.5759,120.0000,600.0000,720.0000,780.0000,6999.0000,0
2,spkts,int64,400000,7.7919,3.5161,1.0000,5.0000,7.0000,11.0000,28.0000,0
3,dpkts,int64,400000,0.3239,0.6746,0.0000,0.0000,0.0000,0.0000,28.0000,0
4,sbytes,int64,400000,679.8861,154.7891,60.0000,600.0000,676.0000,780.0000,5872.0000,0
5,dbytes,int64,400000,22.1874,71.4431,0.0000,0.0000,0.0000,0.0000,1680.0000,0
6,sport,int64,400000,28354.2812,17649.2806,-1.0000,12789.0000,26623.0000,41853.0000,65535.0000,0
7,dport,int64,400000,79.9982,0.3842,-1.0000,80.0000,80.0000,80.0000,80.0000,0
8,dur,float64,400000,20.7962,7.7013,0.0001,13.7081,16.9775,26.8716,1555.6840,0
9,mean,float64,400000,2.4034,1.4654,0.0000,1.3380,2.7047,3.5800,4.9110,0



📖 SEMANTIC ANNOTATIONS FOR KEY NUMERICAL FIELDS

  📌 pkts
     Meaning     : Total packets (src+dst)
     Zero means  : No traffic captured
     Extreme means: High-volume attack
     IDS relevance: HIGH

  📌 bytes
     Meaning     : Total bytes (src+dst)
     Zero means  : No payload
     Extreme means: Data exfiltration / flood
     IDS relevance: HIGH

  📌 spkts
     Meaning     : Src→dst packet count
     Zero means  : One-way: dst only
     Extreme means: Flood from src
     IDS relevance: HIGH

  📌 dpkts
     Meaning     : Dst→src packet count
     Zero means  : No response
     Extreme means: Reverse flood
     IDS relevance: HIGH

  📌 sbytes
     Meaning     : Src→dst byte count
     Zero means  : No src payload
     Extreme means: Large upload
     IDS relevance: HIGH

  📌 dbytes
     Meaning     : Dst→src byte count
     Zero means  : No response payload
     Extreme means: Large download
     IDS relevance: HIGH

  📌 sport
     Meaning     : Source port (-1=ARP)
     Zero m

---
## Section 0.5 — Full Bot-IoT Feature Meaning Dictionary

This section constructs a complete semantic dictionary for all 46 Bot-IoT features.
Each entry includes:
- **description** — what the field captures  
- **data_type** — expected dtype after Phase 1 casting  
- **zero_when** — when zero / -1 occurs (not always missing)  
- **populated_when** — when the field carries meaningful data  
- **behavioral_relevance** — IDS interpretation  
- **ton_iot_equiv** — corresponding column in TON-IoT (cross-dataset bridge)  
- **iot23_equiv** — corresponding column in IoT-23 (cross-dataset bridge)

In [6]:
# ============================================================
# SECTION 0.5: Full Bot-IoT Feature Meaning Dictionary
# ============================================================

BOTIOT_FEATURE_MEANINGS = {
    "pkSeqID": {
        "description": "Monotonically increasing packet/flow sequence ID. Row identifier with no behavioral signal.",
        "data_type": "int64", "zero_when": "Never (starts from 1)",
        "populated_when": "Always", "behavioral_relevance": "NONE — drop before modeling",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "stime": {
        "description": "Flow start time as Unix epoch (float). Used for temporal analysis and train/test splitting.",
        "data_type": "float64", "zero_when": "Never in valid capture",
        "populated_when": "Always", "behavioral_relevance": "MEDIUM — temporal context; exclude from feature vector or bin by hour",
        "ton_iot_equiv": "ts", "iot23_equiv": "ts",
    },
    "ltime": {
        "description": "Flow last-packet time as Unix epoch (float). ltime - stime ≈ dur.",
        "data_type": "float64", "zero_when": "Never in valid capture",
        "populated_when": "Always", "behavioral_relevance": "LOW — redundant with dur; use dur instead",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "flgs": {
        "description": "Argus flow flags string (e.g., 'e', 'e s', '  '). Captures TCP/UDP/ARP control flags in an Argus-encoded compacted form.",
        "data_type": "str/category", "zero_when": "N/A — string field",
        "populated_when": "Always; varies by protocol",
        "behavioral_relevance": "HIGH — encodes SYN/RST/FIN/ACK patterns; use flgs_number or parse flags",
        "ton_iot_equiv": "—", "iot23_equiv": "history (partial analog)",
    },
    "flgs_number": {
        "description": "Numeric encoding of flgs. Integer representation of Argus flag bitmask.",
        "data_type": "int64", "zero_when": "No flags set",
        "populated_when": "Always", "behavioral_relevance": "HIGH — numeric form of flgs; prefer over string flgs for ML",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "proto": {
        "description": "Layer-4 protocol name string: 'tcp', 'udp', 'arp', 'icmp', etc.",
        "data_type": "str/category", "zero_when": "N/A",
        "populated_when": "Always", "behavioral_relevance": "HIGH — encode as ordinal or one-hot; DoS often uses one protocol heavily",
        "ton_iot_equiv": "proto", "iot23_equiv": "proto",
    },
    "proto_number": {
        "description": "Numeric encoding of proto. IANA protocol numbers (TCP=6, UDP=17, ICMP=1, ARP=2).",
        "data_type": "int64", "zero_when": "N/A",
        "populated_when": "Always", "behavioral_relevance": "HIGH — numeric form of proto",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "saddr": {
        "description": "Source IP address string. Identity feature — may cause leakage if used directly.",
        "data_type": "str", "zero_when": "N/A",
        "populated_when": "Always", "behavioral_relevance": "IDENTITY LEAKAGE — use behavioral aggregates (TnBPSrcIP, etc.) instead",
        "ton_iot_equiv": "src_ip", "iot23_equiv": "id.orig_h",
    },
    "sport": {
        "description": "Source port number. -1 for ARP and other non-port protocols.",
        "data_type": "int64", "zero_when": "Port 0 probe",
        "populated_when": "TCP/UDP flows; -1 for ARP/ICMP",
        "behavioral_relevance": "MEDIUM — categorical, NOT ordinal; high ephemeral ports indicate client; low ports may indicate server",
        "ton_iot_equiv": "src_port", "iot23_equiv": "id.orig_p",
    },
    "daddr": {
        "description": "Destination IP address string.",
        "data_type": "str", "zero_when": "N/A",
        "populated_when": "Always", "behavioral_relevance": "IDENTITY LEAKAGE — repeated targets suggest DDoS victim",
        "ton_iot_equiv": "dst_ip", "iot23_equiv": "id.resp_h",
    },
    "dport": {
        "description": "Destination port number. -1 for ARP and other non-port protocols.",
        "data_type": "int64", "zero_when": "Port 0 probe",
        "populated_when": "TCP/UDP flows; -1 for ARP/ICMP",
        "behavioral_relevance": "HIGH — well-known ports (80, 443, 22) vs random ports indicate service targeting",
        "ton_iot_equiv": "dst_port", "iot23_equiv": "id.resp_p",
    },
    "pkts": {
        "description": "Total packet count (spkts + dpkts). High values indicate flood attacks.",
        "data_type": "int64", "zero_when": "No traffic (edge case)",
        "populated_when": "Always", "behavioral_relevance": "HIGH — use log-transform in Phase 1 due to skew",
        "ton_iot_equiv": "—", "iot23_equiv": "orig_pkts + resp_pkts (sum)",
    },
    "bytes": {
        "description": "Total byte count (sbytes + dbytes). Key volume indicator.",
        "data_type": "int64", "zero_when": "No payload (ICMP, ARP)",
        "populated_when": "TCP/UDP data flows", "behavioral_relevance": "HIGH — exfiltration = large bytes; DoS = many pkts with small bytes",
        "ton_iot_equiv": "—", "iot23_equiv": "orig_ip_bytes + resp_ip_bytes (approx)",
    },
    "state": {
        "description": "Argus connection state string (RST, CON, FIN, INT, REQ, ACC, CLO, etc.).",
        "data_type": "str/category", "zero_when": "N/A",
        "populated_when": "Always; varies by protocol",
        "behavioral_relevance": "HIGH — RST-heavy = scan/DoS; CON = established; FIN = clean termination",
        "ton_iot_equiv": "—", "iot23_equiv": "conn_state (similar concept via Zeek)",
    },
    "state_number": {
        "description": "Numeric encoding of state. Prefer over state string for ML.",
        "data_type": "int64", "zero_when": "N/A",
        "populated_when": "Always", "behavioral_relevance": "HIGH — numeric form of state",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "seq": {
        "description": "Argus flow transaction sequence number. Monotonically increases within a capture.",
        "data_type": "int64", "zero_when": "Never",
        "populated_when": "Always", "behavioral_relevance": "LOW — sequence context only; not a behavioral feature",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "dur": {
        "description": "Flow duration in seconds (ltime - stime). Zero for sub-millisecond flows.",
        "data_type": "float64", "zero_when": "Sub-millisecond (ICMP probes, ARP, single-packet)",
        "populated_when": "Always", "behavioral_relevance": "HIGH — very short = probes/scans; very long = exfiltration / C2 channel",
        "ton_iot_equiv": "duration", "iot23_equiv": "duration (stored as string in Zeek)",
    },
    "mean": {
        "description": "Mean of per-packet statistics (IAT or size) over the flow.",
        "data_type": "float64", "zero_when": "Single-packet flow",
        "populated_when": "Multi-packet flows", "behavioral_relevance": "MEDIUM — regular mean = automated traffic; irregular = human-driven",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "stddev": {
        "description": "Standard deviation of per-packet statistics. Zero = perfectly uniform traffic.",
        "data_type": "float64", "zero_when": "Uniform packet sizes",
        "populated_when": "Multi-packet flows", "behavioral_relevance": "MEDIUM — zero stddev in DoS floods is a strong attack indicator",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "sum": {
        "description": "Sum of packet size statistics over the flow. Related to bytes.",
        "data_type": "float64", "zero_when": "No payload",
        "populated_when": "Multi-packet flows", "behavioral_relevance": "LOW — redundant with bytes",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "min": {
        "description": "Minimum packet size in flow.",
        "data_type": "float64", "zero_when": "Zero-byte packet (keep-alive)",
        "populated_when": "Multi-packet flows", "behavioral_relevance": "MEDIUM — min≈max = uniform flood",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "max": {
        "description": "Maximum packet size in flow.",
        "data_type": "float64", "zero_when": "Zero-byte packet",
        "populated_when": "Multi-packet flows", "behavioral_relevance": "MEDIUM — max = MTU-sized = large payload transfer",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "spkts": {
        "description": "Source-to-destination packet count.",
        "data_type": "int64", "zero_when": "No src packets (dst-only flow)",
        "populated_when": "Most flows", "behavioral_relevance": "HIGH — asymmetric spkts≫dpkts = one-way flood",
        "ton_iot_equiv": "—", "iot23_equiv": "orig_pkts",
    },
    "dpkts": {
        "description": "Destination-to-source packet count.",
        "data_type": "int64", "zero_when": "No response (SYN scan, ICMP probe)",
        "populated_when": "Bidirectional flows", "behavioral_relevance": "HIGH — zero dpkts = unanswered scan",
        "ton_iot_equiv": "—", "iot23_equiv": "resp_pkts",
    },
    "sbytes": {
        "description": "Source-to-destination byte count.",
        "data_type": "int64", "zero_when": "No src payload",
        "populated_when": "Most TCP/UDP flows", "behavioral_relevance": "HIGH — large sbytes = upload / exfiltration",
        "ton_iot_equiv": "—", "iot23_equiv": "orig_ip_bytes",
    },
    "dbytes": {
        "description": "Destination-to-source byte count.",
        "data_type": "int64", "zero_when": "No response / one-way",
        "populated_when": "Bidirectional flows", "behavioral_relevance": "HIGH — zero dbytes + high sbytes = attack probe / flood",
        "ton_iot_equiv": "—", "iot23_equiv": "resp_ip_bytes",
    },
    "rate": {
        "description": "Overall flow rate in bytes/second. rate = bytes / dur.",
        "data_type": "float64", "zero_when": "Zero bytes or zero duration",
        "populated_when": "Most flows", "behavioral_relevance": "HIGH — extreme rate = DDoS; log-transform required",
        "ton_iot_equiv": "rate", "iot23_equiv": "—",
    },
    "srate": {
        "description": "Source-side flow rate (sbytes / dur).",
        "data_type": "float64", "zero_when": "No src bytes",
        "populated_when": "Most flows", "behavioral_relevance": "HIGH — srate≫drate = upload flood",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "drate": {
        "description": "Destination-side flow rate (dbytes / dur).",
        "data_type": "float64", "zero_when": "No dst bytes",
        "populated_when": "Bidirectional flows", "behavioral_relevance": "HIGH — drate≫srate = download flood / amplification",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "TnBPSrcIP": {
        "description": "Total bytes from source IP in a recent time window. UNSW-engineered stat.",
        "data_type": "float64", "zero_when": "First appearance of this src IP",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — high value = this IP is sending a lot",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "TnBPDstIP": {
        "description": "Total bytes to destination IP in a recent time window.",
        "data_type": "float64", "zero_when": "First appearance of dst IP",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — high value = this IP is being flooded (DDoS victim)",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "TnP_PSrcIP": {
        "description": "Total packets from source IP in a recent time window.",
        "data_type": "float64", "zero_when": "First appearance of src IP",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — high = scanning / DoS src",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "TnP_PDstIP": {
        "description": "Total packets to destination IP in a recent time window.",
        "data_type": "float64", "zero_when": "First appearance of dst IP",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — high = target under attack",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "TnP_PerProto": {
        "description": "Total packets for this protocol in a recent time window. Captures protocol flooding.",
        "data_type": "float64", "zero_when": "Rare protocol",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — extreme value = protocol-specific flooding",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "TnP_Per_Dport": {
        "description": "Total packets to this destination port in a recent time window.",
        "data_type": "float64", "zero_when": "First flow to this port",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — high = port-specific attack (e.g., HTTP flood on port 80)",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "AR_P_Proto_P_SrcIP": {
        "description": "Average rate: protocol × source IP in window.",
        "data_type": "float64", "zero_when": "No prior flows for this proto/src combo",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — captures per-protocol per-source flooding rate",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "AR_P_Proto_P_DstIP": {
        "description": "Average rate: protocol × destination IP in window.",
        "data_type": "float64", "zero_when": "No prior flows for this proto/dst combo",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — captures DDoS victim under specific protocol",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "N_IN_Conn_P_DstIP": {
        "description": "Number of inbound connections to destination IP in window.",
        "data_type": "float64", "zero_when": "First inbound connection to dst",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — extreme = DDoS target",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "N_IN_Conn_P_SrcIP": {
        "description": "Number of inbound connections from source IP in window.",
        "data_type": "float64", "zero_when": "First connection from src",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — extreme = src is a botnet node",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "AR_P_Proto_P_Sport": {
        "description": "Average rate: protocol × source port in window.",
        "data_type": "float64", "zero_when": "No prior flows for this proto/sport combo",
        "populated_when": "All flows", "behavioral_relevance": "MEDIUM — ephemeral ports make this noisier",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "AR_P_Proto_P_Dport": {
        "description": "Average rate: protocol × destination port in window.",
        "data_type": "float64", "zero_when": "No prior flows for this proto/dport combo",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — captures service-specific attack rate",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "Pkts_P_State_P_Protocol_P_DestIP": {
        "description": "Packets with this state × protocol × destination IP in window.",
        "data_type": "float64", "zero_when": "No prior matching flows",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — RST×TCP×specific_IP = targeted reset flooding",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "Pkts_P_State_P_Protocol_P_SrcIP": {
        "description": "Packets with this state × protocol × source IP in window.",
        "data_type": "float64", "zero_when": "No prior matching flows",
        "populated_when": "All flows", "behavioral_relevance": "HIGH — captures stateful pattern per source",
        "ton_iot_equiv": "—", "iot23_equiv": "—",
    },
    "attack": {
        "description": "Binary attack label: 0 = Normal traffic, 1 = Attack traffic.",
        "data_type": "int64", "zero_when": "N/A — label field",
        "populated_when": "Always", "behavioral_relevance": "PRIMARY LABEL — target variable for binary classification",
        "ton_iot_equiv": "label", "iot23_equiv": "label (Malicious/Benign)",
    },
    "category": {
        "description": "Attack category string: 'Normal', 'DoS', 'DDoS', 'Reconnaissance', 'Theft'.",
        "data_type": "str/category", "zero_when": "N/A — label field",
        "populated_when": "Always", "behavioral_relevance": "MULTI-CLASS LABEL — 5 categories; use for multi-class classification",
        "ton_iot_equiv": "type", "iot23_equiv": "detailed-label (partial)",
    },
    "subcategory": {
        "description": "Fine-grained attack subtype: 'HTTP', 'UDP', 'TCP', 'Keylogging', 'Data_Exfiltration', 'OS_Fingerprint', etc.",
        "data_type": "str/category", "zero_when": "N/A — label field",
        "populated_when": "Always", "behavioral_relevance": "FINE-GRAINED LABEL — most specific class; use for hierarchical classification",
        "ton_iot_equiv": "type (partial)", "iot23_equiv": "detailed-label",
    },
}

# ── Display as DataFrame ──────────────────────────────────────
meaning_rows = []
for col, info in BOTIOT_FEATURE_MEANINGS.items():
    meaning_rows.append({
        "column":              col,
        "description":         info["description"][:70],
        "data_type":           info["data_type"],
        "behavioral_relevance":info["behavioral_relevance"][:60],
        "ton_iot_equiv":       info["ton_iot_equiv"],
        "iot23_equiv":         info["iot23_equiv"],
    })

meaning_df = pd.DataFrame(meaning_rows)
print("=" * 65)
print("📚 FULL BOT-IOT FEATURE MEANING TABLE")
print("=" * 65)
display(meaning_df)
print(f"\n  Total features documented: {len(meaning_df)}")

📚 FULL BOT-IOT FEATURE MEANING TABLE


,column,description,data_type,behavioral_relevance,ton_iot_equiv,iot23_equiv
0,pkSeqID,Monotonically increasing packet/flow sequence ID. Row identifier with,int64,NONE — drop before modeling,—,—
1,stime,Flow start time as Unix epoch (float). Used for temporal analysis and,float64,MEDIUM — temporal context; exclude from feature vector or bi,ts,ts
2,ltime,Flow last-packet time as Unix epoch (float). ltime - stime ≈ dur.,float64,LOW — redundant with dur; use dur instead,—,—
3,flgs,"Argus flow flags string (e.g., 'e', 'e s', ' '). Captures TCP/UDP/ARP",str/category,HIGH — encodes SYN/RST/FIN/ACK patterns; use flgs_number or,—,history (partial analog)
4,flgs_number,Numeric encoding of flgs. Integer representation of Argus flag bitmask,int64,HIGH — numeric form of flgs; prefer over string flgs for ML,—,—
5,proto,"Layer-4 protocol name string: 'tcp', 'udp', 'arp', 'icmp', etc.",str/category,HIGH — encode as ordinal or one-hot; DoS often uses one prot,proto,proto
6,proto_number,"Numeric encoding of proto. IANA protocol numbers (TCP=6, UDP=17, ICMP=",int64,HIGH — numeric form of proto,—,—
7,saddr,Source IP address string. Identity feature — may cause leakage if used,str,"IDENTITY LEAKAGE — use behavioral aggregates (TnBPSrcIP, etc",src_ip,id.orig_h
8,sport,Source port number. -1 for ARP and other non-port protocols.,int64,"MEDIUM — categorical, NOT ordinal; high ephemeral ports indi",src_port,id.orig_p
9,daddr,Destination IP address string.,str,IDENTITY LEAKAGE — repeated targets suggest DDoS victim,dst_ip,id.resp_h



  Total features documented: 46


---
## Section 0.6 — Feature Role Classification (IDS Perspective)

Every feature plays a distinct role in an IDS pipeline. This classification drives which features:
- **Enter the model input vector** (Behavioral, Contextual)
- **Are used only for splitting/grouping** (Identifier, Metadata)
- **Are the prediction target** (Label)

| Role | IDS Meaning | Phase 1 Action |
|---|---|---|
| **Behavioral** | Direct signal of attack vs. normal | Include in feature vector |
| **Identifier** | IP / port / ID — causes data leakage | Exclude from feature vector |
| **Contextual** | Protocol / state / flags — categorical encoding needed | Encode and include |
| **Statistical** | UNSW-engineered window aggregates | Include; normalize |
| **Temporal** | Timestamps — for splitting only | Exclude from features; use for split |
| **Label** | Ground truth | Target variable |
| **Metadata** | Source file / sequence — no signal | Drop |

In [7]:
# ============================================================
# SECTION 0.6: Feature Role Classification
# ============================================================

BOTIOT_ROLE_MAP = {
    # ── Identifiers (EXCLUDE from model) ─────────────────────
    "pkSeqID":   "Identifier",
    "saddr":     "Identifier",
    "daddr":     "Identifier",
    "sport":     "Identifier",        # port IS categorical-behavioral but leaks identity
    "dport":     "Identifier",
    # ── Temporal (EXCLUDE from features; use for splitting) ──
    "stime":     "Temporal",
    "ltime":     "Temporal",
    "seq":       "Temporal",
    # ── Contextual (ENCODE and include) ─────────────────────
    "proto":     "Contextual",
    "proto_number": "Contextual",
    "flgs":      "Contextual",
    "flgs_number":  "Contextual",
    "state":     "Contextual",
    "state_number": "Contextual",
    # ── Behavioral — Flow Volume (INCLUDE) ──────────────────
    "pkts":      "Behavioral",
    "bytes":     "Behavioral",
    "spkts":     "Behavioral",
    "dpkts":     "Behavioral",
    "sbytes":    "Behavioral",
    "dbytes":    "Behavioral",
    "dur":       "Behavioral",
    "mean":      "Behavioral",
    "stddev":    "Behavioral",
    "sum":       "Behavioral",
    "min":       "Behavioral",
    "max":       "Behavioral",
    "rate":      "Behavioral",
    "srate":     "Behavioral",
    "drate":     "Behavioral",
    # ── Statistical / UNSW-engineered (INCLUDE; normalize) ──
    "TnBPSrcIP":                   "Statistical",
    "TnBPDstIP":                   "Statistical",
    "TnP_PSrcIP":                  "Statistical",
    "TnP_PDstIP":                  "Statistical",
    "TnP_PerProto":                "Statistical",
    "TnP_Per_Dport":               "Statistical",
    "AR_P_Proto_P_SrcIP":          "Statistical",
    "AR_P_Proto_P_DstIP":          "Statistical",
    "N_IN_Conn_P_DstIP":           "Statistical",
    "N_IN_Conn_P_SrcIP":           "Statistical",
    "AR_P_Proto_P_Sport":          "Statistical",
    "AR_P_Proto_P_Dport":          "Statistical",
    "Pkts_P_State_P_Protocol_P_DestIP": "Statistical",
    "Pkts_P_State_P_Protocol_P_SrcIP":  "Statistical",
    # ── Labels ───────────────────────────────────────────────
    "attack":      "Label",
    "category":    "Label",
    "subcategory": "Label",
    # ── Metadata (DROP) ──────────────────────────────────────
    "source_file": "Metadata",
}

ROLE_NOTES = {
    "Behavioral":  "Direct signal of attack behavior — include in feature vector (normalize in Phase 1)",
    "Identifier":  "IP/sequence — identity leakage risk — exclude from model",
    "Contextual":  "Protocol/state/flags — encode as ordinal/one-hot, then include",
    "Statistical": "UNSW window-aggregated stats — include; likely need log-transform + normalization",
    "Temporal":    "Timestamps — use only for time-based train/test splitting; exclude from features",
    "Label":       "Prediction target — never included in feature vector",
    "Metadata":    "Source provenance — no behavioral signal; drop before training",
}

def classify_role(col):
    return BOTIOT_ROLE_MAP.get(col, "Behavioral")   # default assumption

# ── Build role DataFrame ──────────────────────────────────────
role_rows = []
for col in df.columns:
    role = classify_role(col)
    note = ROLE_NOTES.get(role, "")
    role_rows.append({"column": col, "role": role, "note": note})

role_df = pd.DataFrame(role_rows)

print("=" * 65)
print("🏷️  FEATURE ROLE CLASSIFICATION")
print("=" * 65)
display(role_df)

print("\n" + "=" * 65)
print("📊 ROLE DISTRIBUTION SUMMARY")
print("=" * 65)
role_counts = role_df["role"].value_counts()
for role, cnt in role_counts.items():
    pct   = cnt / len(role_df) * 100
    emoji = {"Behavioral":"🔴","Identifier":"🟡","Contextual":"🟠",
             "Statistical":"🔵","Temporal":"⚫","Label":"🟢","Metadata":"⚪"}.get(role,"⬜")
    cols_in_role = role_df[role_df["role"]==role]["column"].tolist()
    print(f"\n  {emoji} {role:12s} : {cnt:2d} cols ({pct:.0f}%)")
    print(f"     → {cols_in_role}")

🏷️  FEATURE ROLE CLASSIFICATION


,column,role,note
0,pkSeqID,Identifier,IP/sequence — identity leakage risk — exclude from model
1,stime,Temporal,Timestamps — use only for time-based train/test splitting; exclude from feat...
2,flgs,Contextual,"Protocol/state/flags — encode as ordinal/one-hot, then include"
3,flgs_number,Contextual,"Protocol/state/flags — encode as ordinal/one-hot, then include"
4,proto,Contextual,"Protocol/state/flags — encode as ordinal/one-hot, then include"
5,proto_number,Contextual,"Protocol/state/flags — encode as ordinal/one-hot, then include"
6,saddr,Identifier,IP/sequence — identity leakage risk — exclude from model
7,sport,Identifier,IP/sequence — identity leakage risk — exclude from model
8,daddr,Identifier,IP/sequence — identity leakage risk — exclude from model
9,dport,Identifier,IP/sequence — identity leakage risk — exclude from model



📊 ROLE DISTRIBUTION SUMMARY

  🔴 Behavioral   : 15 cols (32%)
     → ['pkts', 'bytes', 'dur', 'mean', 'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate']

  🔵 Statistical  : 14 cols (30%)
     → ['TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP', 'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport', 'Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP']

  🟠 Contextual   :  6 cols (13%)
     → ['flgs', 'flgs_number', 'proto', 'proto_number', 'state', 'state_number']

  🟡 Identifier   :  5 cols (11%)
     → ['pkSeqID', 'saddr', 'sport', 'daddr', 'dport']

  ⚫ Temporal     :  3 cols (6%)
     → ['stime', 'ltime', 'seq']

  🟢 Label        :  3 cols (6%)
     → ['attack', 'category', 'subcategory']

  ⚪ Metadata     :  1 cols (2%)
     → ['source_file']


---
### Tri-Label Taxonomy Analysis

Bot-IoT has a **3-level label hierarchy** — unique among the three datasets:

```
attack (binary: 0/1)
  └── category (5 values: Normal, DoS, DDoS, Reconnaissance, Theft)
        └── subcategory (fine-grained: HTTP, UDP, TCP, Keylogging, Data_Exfiltration, OS_Fingerprint…)
```

This hierarchy enables:
- **Level 1** — binary IDS alert (attack / normal)
- **Level 2** — attack family classification (DoS family vs Recon vs Theft)
- **Level 3** — specific method identification (most useful for SOC analysts)

In [8]:
# ── Tri-Label Taxonomy Analysis ──────────────────────────────
print("=" * 65)
print("🏷️  TRI-LABEL TAXONOMY ANALYSIS")
print("=" * 65)

# Level 1: binary
print("\n📊 Level 1: attack (0=Normal, 1=Attack)")
attack_vc = df["attack"].value_counts()
for val, cnt in attack_vc.items():
    pct = cnt / len(df) * 100
    label = "Normal" if val == 0 else "Attack"
    print(f"   {val} ({label}) : {cnt:>10,}  ({pct:.2f}%)")

# Level 2: category
print("\n📊 Level 2: category")
cat_vc = df["category"].value_counts()
for val, cnt in cat_vc.items():
    pct = cnt / len(df) * 100
    print(f"   {str(val):25s} : {cnt:>10,}  ({pct:.2f}%)")

# Level 3: subcategory
print("\n📊 Level 3: subcategory")
subcat_vc = df["subcategory"].value_counts()
for val, cnt in subcat_vc.items():
    pct = cnt / len(df) * 100
    print(f"   {str(val):30s} : {cnt:>10,}  ({pct:.2f}%)")

# Cross-tab: category × subcategory
print("\n📊 Category × Subcategory cross-tab (counts):")
cross = pd.crosstab(df["category"], df["subcategory"])
display(cross)

# Imbalance check
total = len(df)
normal_cnt = (df["attack"] == 0).sum()
attack_cnt = (df["attack"] == 1).sum()
imbalance_ratio = attack_cnt / max(normal_cnt, 1)

print(f"\n📊 CLASS IMBALANCE SUMMARY")
print(f"   Total samples : {total:,}")
print(f"   Normal (0)    : {normal_cnt:,}  ({normal_cnt/total*100:.2f}%)")
print(f"   Attack (1)    : {attack_cnt:,}  ({attack_cnt/total*100:.2f}%)")
print(f"   Attack/Normal : {imbalance_ratio:.1f}x")
if imbalance_ratio > 10:
    print("   ⚠️  SEVERE imbalance — Phase 1 must use stratified sampling or class-weight correction")
elif imbalance_ratio > 3:
    print("   ⚠️  MODERATE imbalance — Phase 1 should use stratified train/test split")
else:
    print("   ✅ Imbalance within acceptable range")

🏷️  TRI-LABEL TAXONOMY ANALYSIS

📊 Level 1: attack (0=Normal, 1=Attack)
   1 (Attack) :    400,000  (100.00%)

📊 Level 2: category
   DoS                       :    200,000  (50.00%)
   DDoS                      :    200,000  (50.00%)

📊 Level 3: subcategory
   UDP                            :    200,000  (50.00%)
   TCP                            :    198,515  (49.63%)
   HTTP                           :      1,485  (0.37%)

📊 Category × Subcategory cross-tab (counts):


subcategory,HTTP,TCP,UDP
category,,,
DDoS,0,100000,100000
DoS,1485,98515,100000



📊 CLASS IMBALANCE SUMMARY
   Total samples : 400,000
   Normal (0)    : 0  (0.00%)
   Attack (1)    : 400,000  (100.00%)
   Attack/Normal : 400000.0x
   ⚠️  SEVERE imbalance — Phase 1 must use stratified sampling or class-weight correction


---
## Section 0.7 — Engineering Challenges, Artifact Export & Markdown Report

---
## Full Dataset Statistics — ALL Rows (~14M, chunked, no RAM limit)

> The Section 0.1–0.7 EDA above used `MAX_ROWS_PER_FILE = 100,000` for speed and safety.
> Because the Bot-IoT files are **attack-type ordered**, the first 100k rows of each file are
> 100% attack traffic — no Normal records appear. This section performs a **complete scan
> of every row** using chunked I/O (500k rows at a time) to produce accurate dataset-wide statistics.

**Key aggregations computed:**
- Exact row count per file and total
- Full label distributions: `attack` / `category` / `subcategory`
- Full protocol distribution: `proto`
- Full connection state distribution: `state`
- Per-column numerical min / max / mean (running sum/count method, read-only)
- Port -1 sentinel count across all rows

In [12]:
# ============================================================
# FULL DATASET SCAN — ALL ROWS (Bot-IoT, ~14M rows)
# Chunked reading: 500k rows at a time → never loads full data into RAM
# READ-ONLY: df from Phase 0 EDA is not touched; new accumulators only
# ============================================================
import time

FULL_CHUNKSIZE = 500_000

# ── Accumulators ─────────────────────────────────────────────
full_file_rows      = {}          # fname → row count
full_total_rows     = 0

full_attack_counts  = {}          # {0: n, 1: n}
full_cat_counts     = {}          # {category_str: n}
full_subcat_counts  = {}          # {subcategory_str: n}
full_proto_counts   = {}          # {proto_str: n}
full_state_counts   = {}          # {state_str: n}
full_flgs_counts    = {}          # {flgs_str: n}

# Numerical running stats (sum/count/min/max per column)
NUM_COLS_FULL = ["pkts","bytes","spkts","dpkts","sbytes","dbytes",
                 "dur","rate","srate","drate","mean","stddev",
                 "sport","dport","TnBPSrcIP","TnBPDstIP",
                 "N_IN_Conn_P_DstIP","N_IN_Conn_P_SrcIP"]
full_num_stats = {c: {"count": 0, "sum": 0.0, "min": float("inf"), "max": float("-inf")}
                  for c in NUM_COLS_FULL}
full_port_neg1 = 0

def _accum_vc(acc_dict, series):
    """Add a pandas value_counts Series into an accumulator dict."""
    for val, n in series.value_counts().items():
        k = str(val)
        acc_dict[k] = acc_dict.get(k, 0) + int(n)

def _accum_num(stats_dict, col_name, series):
    """Update running min/max/sum/count for a numeric column."""
    s = pd.to_numeric(series, errors="coerce").dropna()
    if len(s) == 0:
        return
    s_stats = stats_dict[col_name]
    s_stats["count"] += len(s)
    s_stats["sum"]   += float(s.sum())
    s_stats["min"]    = min(s_stats["min"], float(s.min()))
    s_stats["max"]    = max(s_stats["max"], float(s.max()))

# ── Main scan loop ────────────────────────────────────────────
print("=" * 65)
print("🔍 FULL DATASET SCAN — ALL ROWS (500k-row chunks)")
print("=" * 65)

t0 = time.time()
for fname in BOT_IOT_FILENAMES:
    fpath  = BASE_DATA_PATH / fname
    f_rows = 0
    print(f"\n  📂 {fname}")

    reader = pd.read_csv(fpath, chunksize=FULL_CHUNKSIZE, low_memory=False)
    for i, chunk in enumerate(reader):
        f_rows += len(chunk)

        # Labels
        _accum_vc(full_attack_counts,  chunk["attack"])
        _accum_vc(full_cat_counts,     chunk["category"])
        _accum_vc(full_subcat_counts,  chunk["subcategory"])
        # Contextual
        _accum_vc(full_proto_counts,   chunk["proto"])
        _accum_vc(full_state_counts,   chunk["state"])
        _accum_vc(full_flgs_counts,    chunk["flgs"])
        # Numerical
        for col in NUM_COLS_FULL:
            if col in chunk.columns:
                _accum_num(full_num_stats, col, chunk[col])
        # Port -1 sentinel
        if "sport" in chunk.columns:
            full_port_neg1 += int((chunk["sport"] == -1).sum())
        if "dport" in chunk.columns:
            full_port_neg1 += int((chunk["dport"] == -1).sum())

        print(f"     chunk {i+1:>3d} — {f_rows:>10,} rows so far", end="\r")

    full_file_rows[fname]  = f_rows
    full_total_rows       += f_rows
    print(f"     ✅ {fname:50s} → {f_rows:>10,} rows total  ")

elapsed = time.time() - t0
print(f"\n{'─'*65}")
print(f"  ✅ Full scan complete in {elapsed:.1f}s")
print(f"  Total rows across all files: {full_total_rows:,}")
print(f"{'─'*65}")

# ── Display: per-file row counts ──────────────────────────────
print("\n📋 PER-FILE ROW COUNTS (all rows):")
for fname, rows in full_file_rows.items():
    print(f"   {fname:50s} : {rows:>12,}")
print(f"   {'TOTAL':50s} : {full_total_rows:>12,}")

# ── Display: Label distributions ─────────────────────────────
print("\n" + "=" * 65)
print("🏷️  FULL LABEL DISTRIBUTIONS (all rows)")
print("=" * 65)

print("\n📊 attack (binary):")
for val in sorted(full_attack_counts.keys()):
    cnt = full_attack_counts[val]
    label = "Normal" if str(val) == "0" else "Attack"
    print(f"   {val} ({label:6s}) : {cnt:>12,}  ({cnt/full_total_rows*100:.4f}%)")

print("\n📊 category:")
for val, cnt in sorted(full_cat_counts.items(), key=lambda x: -x[1]):
    print(f"   {str(val):25s} : {cnt:>12,}  ({cnt/full_total_rows*100:.4f}%)")

print("\n📊 subcategory:")
for val, cnt in sorted(full_subcat_counts.items(), key=lambda x: -x[1]):
    print(f"   {str(val):30s} : {cnt:>12,}  ({cnt/full_total_rows*100:.4f}%)")

# ── Display: Protocol + State ─────────────────────────────────
print("\n" + "=" * 65)
print("🔌 FULL PROTOCOL & STATE DISTRIBUTIONS")
print("=" * 65)

print("\n📊 proto:")
for val, cnt in sorted(full_proto_counts.items(), key=lambda x: -x[1]):
    print(f"   {str(val):15s} : {cnt:>12,}  ({cnt/full_total_rows*100:.4f}%)")

print("\n📊 state:")
for val, cnt in sorted(full_state_counts.items(), key=lambda x: -x[1]):
    print(f"   {str(val):15s} : {cnt:>12,}  ({cnt/full_total_rows*100:.4f}%)")

print(f"\n  Port -1 (ARP/no-port) total: {full_port_neg1:,}")

# ── Display: Full numerical stats ────────────────────────────
print("\n" + "=" * 65)
print("📊 FULL NUMERICAL STATISTICS (all rows, running method)")
print("=" * 65)

full_stats_rows = []
for col, s in full_num_stats.items():
    if s["count"] > 0:
        mean_val = s["sum"] / s["count"]
        full_stats_rows.append({
            "column": col,
            "count":  s["count"],
            "mean":   round(mean_val, 4),
            "min":    round(s["min"], 4),
            "max":    round(s["max"], 4),
        })
full_stats_df = pd.DataFrame(full_stats_rows)
display(full_stats_df)

# ── Imbalance summary (full) ──────────────────────────────────
normal_full  = full_attack_counts.get("0", 0)
attack_full  = full_attack_counts.get("1", 0)
print(f"\n📊 FULL DATASET CLASS IMBALANCE:")
print(f"   Normal (0) : {normal_full:>12,}  ({normal_full/full_total_rows*100:.4f}%)")
print(f"   Attack (1) : {attack_full:>12,}  ({attack_full/full_total_rows*100:.4f}%)")
ratio = attack_full / max(normal_full, 1)
print(f"   Attack : Normal ratio = {ratio:.1f}x")

# ── Export full-scan artifacts ────────────────────────────────
full_stats_df.to_csv(artifacts_dir / "botiot_fullscan_numerical_stats.csv", index=False)

full_label_df = pd.DataFrame([
    {"label_level": "attack",      "value": k, "count": v, "pct": round(v/full_total_rows*100, 4)}
    for k, v in full_attack_counts.items()
] + [
    {"label_level": "category",    "value": k, "count": v, "pct": round(v/full_total_rows*100, 4)}
    for k, v in full_cat_counts.items()
] + [
    {"label_level": "subcategory", "value": k, "count": v, "pct": round(v/full_total_rows*100, 4)}
    for k, v in full_subcat_counts.items()
])
full_label_df.to_csv(artifacts_dir / "botiot_fullscan_label_distribution.csv", index=False)

full_row_df = pd.DataFrame([{"file": k, "total_rows": v} for k, v in full_file_rows.items()])
full_row_df.loc[len(full_row_df)] = {"file": "TOTAL", "total_rows": full_total_rows}
full_row_df.to_csv(artifacts_dir / "botiot_fullscan_row_counts.csv", index=False)

print(f"\n✅ Full-scan artifacts saved:")
print(f"   • botiot_fullscan_numerical_stats.csv")
print(f"   • botiot_fullscan_label_distribution.csv")
print(f"   • botiot_fullscan_row_counts.csv")

🔍 FULL DATASET SCAN — ALL ROWS (500k-row chunks)

  📂 UNSW_2018_IoT_Botnet_Full5pc_1.csv
     ✅ UNSW_2018_IoT_Botnet_Full5pc_1.csv                 →  1,000,000 rows total  

  📂 UNSW_2018_IoT_Botnet_Full5pc_2.csv
     ✅ UNSW_2018_IoT_Botnet_Full5pc_2.csv                 →  1,000,000 rows total  

  📂 UNSW_2018_IoT_Botnet_Full5pc_3.csv
     ✅ UNSW_2018_IoT_Botnet_Full5pc_3.csv                 →  1,000,000 rows total  

  📂 UNSW_2018_IoT_Botnet_Full5pc_4.csv
     ✅ UNSW_2018_IoT_Botnet_Full5pc_4.csv                 →    668,522 rows total  

─────────────────────────────────────────────────────────────────
  ✅ Full scan complete in 14.9s
  Total rows across all files: 3,668,522
─────────────────────────────────────────────────────────────────

📋 PER-FILE ROW COUNTS (all rows):
   UNSW_2018_IoT_Botnet_Full5pc_1.csv                 :    1,000,000
   UNSW_2018_IoT_Botnet_Full5pc_2.csv                 :    1,000,000
   UNSW_2018_IoT_Botnet_Full5pc_3.csv                 :    1,000,000
   UNSW

,column,count,mean,min,max
0,pkts,3668522,7.7260,1.0000,70057.0000
1,bytes,3668522,869.0501,60.0000,71833345.0000
2,spkts,3668522,7.3141,1.0000,35029.0000
3,dpkts,3668522,0.4118,0.0000,35029.0000
4,sbytes,3668522,766.3916,42.0000,37747017.0000
5,dbytes,3668522,102.6585,0.0000,34181742.0000
6,dur,3668522,20.3348,0.0000,2771.4851
7,rate,3668522,244.9407,0.0000,1000000.0000
8,srate,3668522,2.9551,0.0000,1000000.0000
9,drate,3668522,0.4455,0.0000,58823.5273



📊 FULL DATASET CLASS IMBALANCE:
   Normal (0) :          477  (0.0130%)
   Attack (1) :    3,668,045  (99.9870%)
   Attack : Normal ratio = 7689.8x

✅ Full-scan artifacts saved:
   • botiot_fullscan_numerical_stats.csv
   • botiot_fullscan_label_distribution.csv
   • botiot_fullscan_row_counts.csv


In [9]:
# ============================================================
# SECTION 0.7: Engineering Challenges
# ============================================================

print("=" * 65)
print("⚠️  ENGINEERING CHALLENGES IDENTIFIED IN PHASE 0")
print("=" * 65)

challenges = {}

# ── Challenge 1: Port -1 sentinel ────────────────────────────
port_neg1 = ((df["sport"] == -1) | (df["dport"] == -1)).sum() if "sport" in df.columns else 0
challenges["port_neg1"] = port_neg1 > 0
print(f"\n{'✅' if not challenges['port_neg1'] else '⚠️ '} 1. PORT -1 SENTINEL")
print(f"   sport==-1 or dport==-1 count: {port_neg1:,}")
print(f"   Issue : -1 encodes 'no port' (ARP/ICMP). Cannot scale with real ports.")
print(f"   Fix   : In Phase 1, create a flag column 'has_port' (1=yes/0=no), then")
print(f"           clip port to [0, 65535] or treat as a separate category.")

# ── Challenge 2: Extreme skew in rate/bytes features ─────────
rate_max = pd.to_numeric(df["rate"], errors="coerce").max() if "rate" in df.columns else 0
challenges["skewed_rates"] = rate_max > 1e6
print(f"\n{'✅' if not challenges['skewed_rates'] else '⚠️ '} 2. EXTREME SKEW IN RATE/VOLUME FEATURES")
print(f"   rate max: {rate_max:,.0f}")
print(f"   Issue : DoS/DDoS attacks produce extreme outliers — pure volume features")
print(f"           have distributions spanning many orders of magnitude.")
print(f"   Fix   : Apply log1p transform: np.log1p(df['rate']) in Phase 1.")
print(f"           Use RobustScaler (IQR-based) rather than StandardScaler.")

# ── Challenge 3: Severe class imbalance ──────────────────────
normal_pct = (df["attack"] == 0).mean() * 100 if "attack" in df.columns else 50
challenges["class_imbalance"] = normal_pct < 5 or normal_pct > 95
print(f"\n{'✅' if not challenges['class_imbalance'] else '⚠️ '} 3. CLASS IMBALANCE")
print(f"   Normal %: {normal_pct:.2f}%")
print(f"   Issue : If Normal << Attack (or vice versa), accuracy is misleading.")
print(f"   Fix   : Use stratified train/test split. Report F1, precision, recall.")
print(f"           Consider SMOTE or class_weight='balanced' in classifiers.")

# ── Challenge 4: Identity leakage via IP/port ────────────────
challenges["identity_leakage"] = True
print(f"\n⚠️  4. IDENTITY LEAKAGE (saddr, daddr, sport, dport)")
print(f"   Issue : These columns uniquely identify hosts/services. A model trained on")
print(f"           them would memorize which IPs/ports appear in attacks — not generalize.")
print(f"   Fix   : Exclude saddr, daddr from feature vector. Treat sport/dport as")
print(f"           categorical buckets (0-1023=system, 1024-49151=registered, etc.).")

# ── Challenge 5: UNSW behavioral features — window size unknown ──
challenges["window_bias"] = True
print(f"\n⚠️  5. UNSW WINDOW-AGGREGATED FEATURES (TnBP*, TnP_*, AR_P_*, N_IN_*, Pkts_P_*)")
print(f"   Issue : These 14 features depend on a time window (not documented in the paper).")
print(f"           They encode aggregate behavior — excellent for attacks but introduce")
print(f"           look-ahead bias if the window crosses the train/test boundary.")
print(f"   Fix   : In Phase 1, perform a strict time-ordered split: train on earlier")
print(f"           captures, test on later ones. Never use random split for time-series data.")

# ── Challenge 6: Redundant features (sum, bytes, sbytes+dbytes) ──
challenges["redundant_features"] = True
print(f"\n⚠️  6. POTENTIALLY REDUNDANT FEATURES")
print(f"   bytes ≈ sbytes + dbytes   |  pkts ≈ spkts + dpkts  |  bytes ≈ sum")
print(f"   Issue : Collinear features inflate feature importance and slow convergence.")
print(f"   Fix   : In Phase 1 compute correlation matrix. Drop features with |r|>0.95.")
print(f"           Candidates to potentially drop: sum, ltime (≈stime+dur).")

# ── Challenge 7: Tri-label consistency ───────────────────────
challenges["label_consistency"] = True
print(f"\n⚠️  7. TRI-LABEL CONSISTENCY")
print(f"   Issue : attack=0 must always pair with category='Normal' and")
print(f"           subcategory='Normal'. Violations indicate labeling errors.")
print(f"   Fix   : In Phase 1, assert: df[df['attack']==0]['category'].eq('Normal').all()")
print(f"           Log and investigate any violations before training.")

print(f"\n{'='*65}")
print(f"  Challenges flagged: {sum(challenges.values())} / {len(challenges)}")
print(f"{'='*65}")

⚠️  ENGINEERING CHALLENGES IDENTIFIED IN PHASE 0

⚠️  1. PORT -1 SENTINEL
   sport==-1 or dport==-1 count: 9
   Issue : -1 encodes 'no port' (ARP/ICMP). Cannot scale with real ports.
   Fix   : In Phase 1, create a flag column 'has_port' (1=yes/0=no), then
           clip port to [0, 65535] or treat as a separate category.

✅ 2. EXTREME SKEW IN RATE/VOLUME FEATURES
   rate max: 7,874
   Issue : DoS/DDoS attacks produce extreme outliers — pure volume features
           have distributions spanning many orders of magnitude.
   Fix   : Apply log1p transform: np.log1p(df['rate']) in Phase 1.
           Use RobustScaler (IQR-based) rather than StandardScaler.

⚠️  3. CLASS IMBALANCE
   Normal %: 0.00%
   Issue : If Normal << Attack (or vice versa), accuracy is misleading.
   Fix   : Use stratified train/test split. Report F1, precision, recall.
           Consider SMOTE or class_weight='balanced' in classifiers.

⚠️  4. IDENTITY LEAKAGE (saddr, daddr, sport, dport)
   Issue : These columns 

In [11]:
# ============================================================
# SECTION 0.7: Artifact Export + Markdown Report Generator
# ============================================================

artifacts_dir = NOTEBOOK_DIR.parent / "artifacts"
artifacts_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Artifacts directory: {artifacts_dir}\n")

# ── Export CSVs ───────────────────────────────────────────────
exports = {
    "column_inventory":   col_inventory,
    "sentinel_analysis":  sentinel_df,
    "feature_meanings":   meaning_df,
    "role_classification": role_df,
    "numerical_stats":    num_stats_df,
}

print("✅ CSV Artifacts saved:")
for key, frame in exports.items():
    fname = f"botiot_phase0_{key}.csv"
    fpath = artifacts_dir / fname
    frame.to_csv(fpath, index=False)
    print(f"   • {key}: {fname}")

# ── File inventory export ─────────────────────────────────────
inv_save = inventory_df.copy()
inv_save.to_csv(artifacts_dir / "botiot_phase0_file_inventory.csv", index=False)
print(f"   • file_inventory: botiot_phase0_file_inventory.csv")

# ── Markdown Report Generator ─────────────────────────────────
def generate_botiot_markdown_report() -> str:
    lines = []
    lines.append("# Phase 0.3 — Bot-IoT Data Understanding Report\n")
    lines.append(f"**Generated:** Phase 0 Read-Only EDA | Rows sampled: {len(df):,} "
                 f"(100k/file × {len(frames)} files)\n")

    lines.append("\n## 1. File Inventory\n")
    lines.append(inventory_df[["file","size_mb"]].to_markdown(index=False))

    lines.append("\n\n## 2. Dataset Shape\n")
    lines.append(f"- **Rows (sampled):** {len(df):,}\n")
    lines.append(f"- **Columns:** {len(df.columns) - 1} (excl. source_file)\n")
    lines.append(f"- **Memory:** {mem_mb:.1f} MB\n")

    lines.append("\n## 3. Column Inventory (dtype, null%, unique)\n")
    lines.append(col_inventory[["column","dtype","null_%","unique"]].to_markdown(index=False))

    lines.append("\n\n## 4. Sentinel Analysis\n")
    lines.append(sentinel_df.to_markdown(index=False))

    lines.append("\n\n## 5. Numerical Statistics\n")
    lines.append(num_stats_df[["column","mean","std","min","median","max"]].to_markdown(index=False))

    lines.append("\n\n## 6. Label Distribution\n")
    lines.append(f"### Binary (attack)\n")
    for val, cnt in df["attack"].value_counts().items():
        lines.append(f"- {val} ({'Normal' if val==0 else 'Attack'}): {cnt:,} ({cnt/len(df)*100:.2f}%)\n")
    lines.append(f"\n### Category\n")
    for val, cnt in df["category"].value_counts().items():
        lines.append(f"- {val}: {cnt:,} ({cnt/len(df)*100:.2f}%)\n")
    lines.append(f"\n### Subcategory\n")
    for val, cnt in df["subcategory"].value_counts().items():
        lines.append(f"- {val}: {cnt:,} ({cnt/len(df)*100:.2f}%)\n")

    lines.append("\n## 7. Feature Role Classification\n")
    lines.append(role_df[["column","role"]].to_markdown(index=False))

    lines.append("\n\n## 8. Engineering Challenges\n")
    challenge_list = [
        ("Port -1 Sentinel", "sport/dport == -1 for ARP. Create has_port flag; clip to [0,65535]."),
        ("Extreme Rate Skew", "rate/srate/drate span orders of magnitude. Apply log1p; use RobustScaler."),
        ("Class Imbalance", "Check attack:normal ratio. Use stratified split + F1/precision/recall."),
        ("Identity Leakage", "saddr/daddr uniquely identify hosts. Exclude from feature vector."),
        ("Window Bias (UNSW features)", "14 window-aggregated features require time-ordered train/test split."),
        ("Redundant Features", "bytes≈sbytes+dbytes, pkts≈spkts+dpkts. Drop |r|>0.95 pairs in Phase 1."),
        ("Tri-Label Consistency", "attack=0 must pair with category=Normal. Assert and log violations."),
    ]
    for name, fix in challenge_list:
        lines.append(f"- **{name}:** {fix}\n")

    lines.append("\n## 9. Cross-Dataset Feature Bridge\n")
    bridge_cols = ["column","ton_iot_equiv","iot23_equiv"]
    bridge_df   = meaning_df[bridge_cols].copy()
    lines.append(bridge_df.to_markdown(index=False))

    lines.append("\n\n---\n*Phase 0 complete — all findings are read-only observations. "
                 "No data was modified.*\n")
    return "".join(lines)

md_report = generate_botiot_markdown_report()
report_path = artifacts_dir / "Phase_0_3_Bot_IoT_Data_Understanding_Report.md"
with open(report_path, "w", encoding="utf-8") as f:
    f.write(md_report)

# Avoid pandas .sum() warning on full df here — use explicit len
print(f"\n📝 Markdown report saved: {report_path.name}")
print(f"   Size: {len(md_report):,} characters / {md_report.count(chr(10))} lines")
print(f"\n{'='*65}")
print(f"✅ PHASE-0.3 COMPLETE — All artifacts exported")
print(f"{'='*65}")

📁 Artifacts directory: C:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\artifacts

✅ CSV Artifacts saved:
   • column_inventory: botiot_phase0_column_inventory.csv
   • sentinel_analysis: botiot_phase0_sentinel_analysis.csv
   • feature_meanings: botiot_phase0_feature_meanings.csv
   • role_classification: botiot_phase0_role_classification.csv
   • numerical_stats: botiot_phase0_numerical_stats.csv
   • file_inventory: botiot_phase0_file_inventory.csv

📝 Markdown report saved: Phase_0_3_Bot_IoT_Data_Understanding_Report.md
   Size: 17,132 characters / 242 lines

✅ PHASE-0.3 COMPLETE — All artifacts exported


---
## Phase 0.3 Complete ✅

| Section | Status | Key Deliverable |
|---|---|---|
| **0.1** File Inventory & Loading | ✅ | `botiot_phase0_file_inventory.csv` |
| **0.2** Column Inventory | ✅ | `botiot_phase0_column_inventory.csv` |
| **0.3** Categorical + Sentinel Analysis | ✅ | `botiot_phase0_sentinel_analysis.csv` |
| **0.4** Numerical Statistics | ✅ | `botiot_phase0_numerical_stats.csv` |
| **0.5** Feature Meaning Dictionary | ✅ | `botiot_phase0_feature_meanings.csv` |
| **0.6** Feature Role Classification | ✅ | `botiot_phase0_role_classification.csv` |
| **0.7** Challenges + Artifacts + Report | ✅ | `Phase_0_3_Bot_IoT_Data_Understanding_Report.md` |

### Key Phase 0 Findings

| Finding | Detail |
|---|---|
| **No Zeek sentinels** | Standard CSV — no `-` or `?`. Sentinel is `-1` for port in ARP flows. |
| **Tri-label hierarchy** | attack (binary) → category (5) → subcategory (fine-grained) — richest label set |
| **14 UNSW behavioral features** | Window-aggregated stats (TnBP*, TnP_*, AR_P_*, N_IN_*, Pkts_P_*) |
| **Identity leakage risk** | saddr, daddr must be excluded from feature vector |
| **Skewed volume features** | rate, bytes, sbytes, dbytes — log1p transform required in Phase 1 |
| **Port -1 anomaly** | ARP/ICMP flows set sport/dport=-1 — create has_port indicator in Phase 1 |
| **Time-ordered split required** | UNSW window features encode temporal state — random split would leak |
| **Next notebook** | `Phase_0_4_Combined_Dataset_Analysis.ipynb` or `Phase_1_Preprocessing.ipynb` |